Создание, обновление и удаление таблиц

1.Создать новую таблицу с разобранными данными таблицы pvp_log. Поля новой таблицы: time, charid, charlevel, actionname, kill, assist, death. Таблицу назвать pvp_log_[ваша фамилия]. Тип столбцов задать исходя из вашего понимания их назначения и дальнейших указаний. Таблицу заполнить данными из запроса к таблице pvp_log. В запросе разобрать поле actiondata на отдельные поля, как вы делали это в задании к четвёртому уроку. В итоге должна получиться таблица, аналогичная таблице pvp_log, но с разобранным полем actiondata. С такой таблицей работать значительно проще.
2.Продемонстрировать запрос на вывод информации о столбцах в таблице из первого пункта.
3.Сохранить в dataframe таблицу со всеми уникальными парами actiontype и actionname из таблицы dungeon_log. Из полученного dataframe создать таблицу в БД с названием dungeon_type_[ваша фамилия]. 4.Продемонстрировать запрос на выборку пяти строк из данной таблицы.
5.Изменить названия столбцов таблицы из пункта 3 на ваше усмотрение. Продемонстрировать изменения в запросе на выборку.
6.Удалить созданные в рамках ДЗ таблицы, продемонстрировать их отсутствие в схеме. Не удалите ничего лишнего!

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'

from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

group_schema = 'gu_game_analytics_1188'

In [3]:
import pandas as pd
import matplotlib

df = pd.read_sql(f'''
SELECT *
FROM {group_schema}.shop_log
''', con=engine)
df

,time,accountid,charid,itemname,itemprice,itemcount
0,2018-08-01 00:03:10.088,492169036,5706508,Малый набор для усиления,500,1
1,2018-08-01 00:09:41.799,909385150,5728727,Малый набор для усиления,500,1
2,2018-08-01 00:20:06.895,461499108,2867292,Премиум статус,1600,1
3,2018-08-01 00:31:06.911,903420362,5263550,Малый набор для усиления,500,1
4,2018-08-01 01:19:21.731,464491245,5738381,Малый набор для усиления,500,1
...,...,...,...,...,...,...
9247,2018-08-31 23:29:03.070,806538906,5715195,Малый набор для усиления,500,1
9248,2018-08-31 23:29:20.216,498324939,5549884,Малый набор для усиления,500,1
9249,2018-08-31 23:44:16.294,489632677,3090883,Малый набор для усиления,500,2
9250,2018-08-31 23:44:45.297,474992099,4874646,Малый набор для усиления,500,2


In [4]:
with engine.connect() as conn:
    conn.execute(f'''
CREATE TABLE {group_schema}.pvp_log_kuznetsov(
time date, 
charid int, 
charlevel int, 
actionname text, 
kill int, 
assist int, 
death int
)
''')


In [5]:
with engine.connect() as conn:
    conn.execute(f'''
INSERT INTO {group_schema}.pvp_log_kuznetsov
SELECT time, charid, charlevel,
    substring(actiondata, 'ActionName\":\"(\w+)') as actionname,
    cast(substring(actiondata, 'Kill\":(\d+)') as INT) as kill,
    cast(substring(actiondata, 'Death\":(\d+)') as INT) as death,
    cast(substring(actiondata, 'Assist\":(\d+)') as INT) as assist
    FROM {group_schema}.pvp_log
''')

In [6]:
df = pd.read_sql(f'''
SELECT *
FROM {group_schema}.pvp_log_kuznetsov
''', con=engine)
df

,time,charid,charlevel,actionname,kill,assist,death
0,2018-08-02,4731904,15,QuickMatch,3,2,3
1,2018-08-07,4731904,16,QuickMatch,4,2,4
2,2018-08-01,4731904,15,QuickMatch,2,3,4
3,2018-08-02,4731904,15,QuickMatch,2,3,2
4,2018-08-03,4731904,16,QuickMatch,2,3,1
...,...,...,...,...,...,...,...
549860,2018-08-03,4731904,16,QuickMatch,3,4,2
549861,2018-08-02,4731904,15,QuickMatch,4,1,5
549862,2018-08-03,4731904,16,QuickMatch,3,4,3
549863,2018-08-04,4731904,16,QuickMatch,2,2,0


Знаю что тип данных в time не такой как надо, но когда я создавал таблицу с типом данных в столбе time: datetime либо datetime2, выдавал странную ошибку, не смог разобраться. Ему либо библиотеки какий-то не хватало, либо еще что...

In [7]:
df = pd.read_sql(f'''
SELECT *
FROM information_schema.columns
WHERE table_name='pvp_log_kuznetsov'
''', con=engine)
df


,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,game-analytics,gu_game_analytics_1188,pvp_log_kuznetsov,time,1,None,YES,date,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,game-analytics,gu_game_analytics_1188,pvp_log_kuznetsov,charid,2,None,YES,integer,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,game-analytics,gu_game_analytics_1188,pvp_log_kuznetsov,charlevel,3,None,YES,integer,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,game-analytics,gu_game_analytics_1188,pvp_log_kuznetsov,actionname,4,None,YES,text,None,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,game-analytics,gu_game_analytics_1188,pvp_log_kuznetsov,kill,5,None,YES,integer,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,game-analytics,gu_game_analytics_1188,pvp_log_kuznetsov,assist,6,None,YES,integer,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
6,game-analytics,gu_game_analytics_1188,pvp_log_kuznetsov,death,7,None,YES,integer,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [8]:
df = pd.read_sql(f'''
SELECT distinct actiontype, actionname
FROM {group_schema}.dungeon_log
''', con=engine)
df

,actiontype,actionname
0,Dungeon,Консерватория грифонов
1,Dungeon,Нага-банк
2,Dungeon,Улей змиев
3,Dungeon,Сокровищница гномов
4,Dungeon,Тайник бесов
5,Raid,Пирамида
6,Raid,Утопия драконов


In [9]:
dungeon_type_kuznetsov = pd.read_sql(f'''
SELECT DISTINCT actiontype, actionname
FROM {group_schema}.dungeon_log
''', con=engine)
dungeon_type_kuznetsov

,actiontype,actionname
0,Dungeon,Консерватория грифонов
1,Dungeon,Нага-банк
2,Dungeon,Улей змиев
3,Dungeon,Сокровищница гномов
4,Dungeon,Тайник бесов
5,Raid,Пирамида
6,Raid,Утопия драконов


In [10]:
dungeon_type_kuznetsov.to_sql(
name='dungeon_type_kuznetsov',
schema=group_schema,
con=engine,
index=False,
if_exists='replace'
)

In [11]:
kuznetsov = pd.read_sql(f'''
SELECT * 
FROM {group_schema}.dungeon_type_kuznetsov 
LIMIT 5
''', con=engine)
kuznetsov


,actiontype,actionname
0,Dungeon,Консерватория грифонов
1,Dungeon,Нага-банк
2,Dungeon,Улей змиев
3,Dungeon,Сокровищница гномов
4,Dungeon,Тайник бесов


In [12]:
df = pd.read_sql(f'''
SELECT *
FROM information_schema.columns
WHERE table_name='dungeon_type_kuznetsov'
''', con=engine)
df


,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,game-analytics,gu_game_analytics_1188,dungeon_type_kuznetsov,actiontype,1,None,YES,text,None,1073741824,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,game-analytics,gu_game_analytics_1188,dungeon_type_kuznetsov,actionname,2,None,YES,text,None,1073741824,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [13]:
with engine.connect() as conn:
    conn.execute(f'''
UPDATE {group_schema}.dungeon_type_kuznetsov
SET actionname = 'Улей пчел' 
WHERE actionname = 'Улей змиев'
''')


Немного не то задание сделал, последний пункт домашки сначала не так понял, но будет как бонус что я разобрался как пользоваться UPDATE ))) 

In [14]:
with engine.connect() as conn:
    conn.execute(f'''
ALTER TABLE {group_schema}.dungeon_type_kuznetsov 
RENAME COLUMN actiontype TO type;
''')


In [15]:
with engine.connect() as conn:
    conn.execute(f'''
ALTER TABLE {group_schema}.dungeon_type_kuznetsov 
RENAME COLUMN actionname TO name;
''')


In [16]:
kuznetsov = pd.read_sql(f'''
SELECT * 
FROM {group_schema}.dungeon_type_kuznetsov 
''', con=engine)
kuznetsov


,type,name
0,Dungeon,Консерватория грифонов
1,Dungeon,Нага-банк
2,Dungeon,Сокровищница гномов
3,Dungeon,Тайник бесов
4,Raid,Пирамида
5,Raid,Утопия драконов
6,Dungeon,Улей пчел


Даже буду знать что после UPDATE данные опускаются в самый низ

In [17]:
with engine.connect() as conn:
    conn.execute(f'''
DROP TABLE IF EXISTS {group_schema}.pvp_log_kuznetsov;
DROP TABLE IF EXISTS {group_schema}.dungeon_type_kuznetsov;
''')